In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 10.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [4]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [5]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")

Langfuse environment variables set successfully!


In [6]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [7]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

In [8]:
task = """Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations"""

In [9]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = ""

In [10]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [12]:
!pip install markdownify requests

In [13]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  locations_info = web_search(query="Harry potter filming locations", filter_year=2020)                            
  locations_info = locations_info.split('\n')  # Split the results into lines                                      
  print('Locations info:', locations_info)                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Locations info: ['## Search Results', '0. [Hogwarts Filming Locations in 
England](https://www.thecravetraveler.com/hogwarts-filming-locations/)', 'Date published: Jan 29, 2020', 'Source: 
The Crave Traveler', '', 'Hogwarts Filming Locations in England: Alnwick Castle, Durham Cathedral, Gloucester 
Cathedral, Lacock Abbey, Oxford, Christ Church, New College, The Divinity ...', '', "1. [Top Sights Where 'Harry 
Potter' Was Filmed](https://www.ricksteves.com/watch-read-listen/read/articles/harry-potter-sites)", 'Date 
published: Jun 30, 2020', 'Source: Rick Steves Europe', '', "Top Sights Where 'Harry Potter' Was Filmed · London · 
Near Bath · Near Brighton · Oxford · Durham and Northeast England · Scottish Highlands · Ireland.", '', '2. [Harry 
Potter ALL London Film Locations - A Free Magic ...](https://www.youtube.com/watch?v=n73Go0Xz8bY)', 'Source: 
YouTube · Svenywhere: Travel Exploration', '', 'Other film locations in London that we cover are London Zoo, 
Lambeth Bridge and the Ministry of Magic area. So join me on a magical walking tour across London ...', '', '3. 
[Filming Locations · Beyond Hogwarts: Harry Potter in 
...](https://lis5472.cci.fsu.edu/sp20/group1/filming-locations)', 'Date published: Jun 14, 2020', 'Source: Florida 
State University', '', 'The Harry Potter movies were filmed throughout England, Scotland, and Wales in the United 
Kingdom. Take a trip on the Jacobite Steam Train in Scotland.', '', '4. [Harry Potter Filming Locations in 
Ireland](https://wildrovertours.com/blog/harry-potter-filming-locations-in-ireland/)', 'Date published: Oct 8, 
2020', 'Source: Wild Rover Tours', '', 'The second Harry Potter filming location in Ireland is Lemon Rock, found 
off the coast County Kerry. Located on the stunning Iveragh peninsula, Lemon Rock is ...', '', '5. [The Ultimate 
Harry Potter Road Trip Across England](https://www.thecravetraveler.com/harry-potter-road-trip/)', 'Date published:
Feb 23, 2020', 'Source: The Crave Traveler', '', "If you're a HUGE Harry Potter fan then this is the perfect Harry 
Potter road trip itinerary for checking out the best filmling locations around England!", '', '6. [Harry Potter 
Locations in Edinburgh that Muggles will 
love](https://www.haggisadventures.com/blog/harry-potter-locations-in-edinburgh-that-muggles-will-love)', 'Date 
published: Jan 23, 2020', 'Source: Haggis Adventures', '', 'Here you can visit Hogwarts Castle, or at least the 
parts of Alnwick Castle which were used for scenes of Hogwarts Castle in the first two Harry Potter films!', '', 
'7. [Visiting Harry Potter Filming Locations in 
Scotland](https://www.rachelnicole.co.uk/2020/10/visiting-harry-potter-filming-locations-in-scotland.html)', 'Date 
published: Oct 24, 2020', 'Source: rachelnicole.co.uk', '', 'Visiting Harry Potter Filming Locations in Scotland · 
Loch Shiel · Loch Elit · Glen Coe · Elephant House Cafe, Edinburgh · Greyfriars Kirkyard, Edinburgh.', '', '8. [15 
Must-Visit HARRY POTTER Filming Locations In 
...](https://londondreaming.com/harry-potter-filming-locations-in-london/)', 'Date published: Sep 9, 2020', 
'Source: London Dreaming', '', "Harry Potter film locations in London · The London Zoo · Leadenhall Market (Harry 
Potter's Diagon Alley Scene, London) · Lambeth Bridge London · Borough Market ...", '', '9. [Top 7 Harry Potter 
Filming Locations 
Blog](https://www.scottishtours.co.uk/blog/top-7-harry-potter-filming-locations-in-scotland-and-england/)', 'Date 
published: Aug 20, 2020', 'Source: Scottish Tours', '', "Edinburgh · Victoria Street · Greyfriar's Kirkyard · 
George Heriot's School · The Elephant House Café and Spoon Café. JK Rowling hung out in The Elephant House ..."]

Out: None

[Step 1: Duration 4.05 seconds| Input tokens: 2,236 | Output tokens: 198]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Manually curated nearby cricket stadiums based on the web search results                                       
  nearby_stadiums = {                                                                                              
      "Alnwick Castle": "Bamburgh Cricket Club",                                                                   
      "Durham Cathedral": "Emirates Riverside (Durham)",                                                           
      "Gloucester Cathedral": "St George's Park, Gloucester",                                                      
      "Lacock Abbey": "Salisbury Cricket Club",                                                                    
      "Christ Church": "University Parks Cricket Club, Oxford",                                                    
      "New College": "University Parks Cricket Club, Oxford",                                                      
      "The Divinity School": "University Parks Cricket Club, Oxford",                                              
      "Duke Humfrey's Library": "University Parks Cricket Club, Oxford",                                           
      "London Zoo": "Lord's Cricket Ground",                                                                       
      "Leadenhall Market": "Lord's Cricket Ground",                                                                
      "Platform 9¾": "Lord's Cricket Ground",                                                                      
      "Westminster Underground Station": "Lord's Cricket Ground",                                                  
      "City Hall": "Lord's Cricket Ground",                                                                        
      "Millennium Bridge": "Lord's Cricket Ground",                                                                
      "Piccadilly Circus": "Lord's Cricket Ground",                                                                
      "Whitehall": "Lord's Cricket Ground",                                                                        
      "Seven Sisters chalk cliffs": "The County Ground, Eastbourne",                                               
      "Glen Coe": "Forfar Cricket Ground",                                                                         
      "Greyfriars Kirkyard": "Grange Cricket Club, Edinburgh",                                                     
      "Edinburgh": "Grange Cricket Club, Edinburgh",                                                               
      "Cliffs of Moher": "Malahide Cricket Club, Dublin"                                                           
  }                                                                                                                
                                                                                                                   
  # Coordinates of the filming locations                                                                           
  locations = {                                                                                                    
      "Alnwick Castle": (55.2466, -2.1790),                                                                        
      "Durham Cathedral": (54.7737, -1.5727),                                                                      
      "Gloucester Cathedral": (51.8582, -2.2876),                                                                  
      "Lacock Abbey": (51.2532, -1.6179),                                                                          
      "[38;2;230;219;116;48;2;39;40;

Execution logs:
Travel time from Pune to Alnwick Castle: 11.92 hours
Travel time from Pune to Durham Cathedral: 11.86 hours
Travel time from Pune to Gloucester Cathedral: 11.93 hours
Travel time from Pune to Lacock Abbey: 11.87 hours
Travel time from Pune to Christ Church: 11.83 hours
Travel time from Pune to New College: 11.83 hours
Travel time from Pune to The Divinity School: 11.83 hours
Travel time from Pune to Duke Humfrey's Library: 11.83 hours
Travel time from Pune to London Zoo: 11.72 hours
Travel time from Pune to Leadenhall Market: 11.71 hours
Travel time from Pune to Platform 9¾: 11.72 hours
Travel time from Pune to Westminster Underground Station: 11.72 hours
Travel time from Pune to City Hall: 11.72 hours
Travel time from Pune to Millennium Bridge: 11.71 hours
Travel time from Pune to Piccadilly Circus: 11.72 hours
Travel time from Pune to Whitehall: 11.72 hours
Travel time from Pune to Seven Sisters chalk cliffs: 12.24 hours
Travel time from Pune to Glen Coe: 12.18 hours
Travel time from Pune to Greyfriars Kirkyard: 12.01 hours
Travel time from Pune to Edinburgh: 12.01 hours
Travel time from Pune to Cliffs of Moher: 12.59 hours
                           Location  Latitude  Longitude  \
0                    Alnwick Castle   55.2466    -2.1790   
1                  Durham Cathedral   54.7737    -1.5727   
2              Gloucester Cathedral   51.8582    -2.2876   
3                      Lacock Abbey   51.2532    -1.6179   
4                     Christ Church   51.7567    -1.2580   
5                       New College   51.7544    -1.2489   
6               The Divinity School   51.7616    -1.2460   
7            Duke Humfrey's Library   51.7574    -1.2548   
8                        London Zoo   51.5373    -0.1419   
9                 Leadenhall Market   51.5121    -0.0945   
10                      Platform 9¾   51.5294    -0.1240   
11  Westminster Underground Station   51.5004    -0.1279   
12                        City Hall   51.5010    -0.1256   
13                Millennium Bridge   51.5038    -0.0988   
14                Piccadilly Circus   51.5108    -0.1403   
15                        Whitehall   51.5033    -0.1279   
16       Seven Sisters chalk cliffs   50.7917    -5.1965   
17                         Glen Coe   56.2514    -5.0513   
18              Greyfriars Kirkyard   55.9480    -3.1919   
19                        Edinburgh   55.9619    -3.1896   
20                  Cliffs of Moher   53.1496    -9.0605   

    Travel Time from Pune                 Nearby Cricket Stadium  
0                   11.92                  Bamburgh Cricket Club  
1                   11.86            Emirates Riverside (Durham)  
2                   11.93           St George's Park, Gloucester  
3                   11.87                 Salisbury Cricket Club  
4                   11.83  University Parks Cricket Club, Oxford  
5                   11.83  University Parks Cricket Club, Oxford  
6                   11.83  University Parks Cricket Club, Oxford  
7                   11.83  University Parks Cricket Club, Oxford  
8                   11.72                  Lord's Cricket Ground  
9                   11.71                  Lord's Cricket Ground  
10                  11.72                  Lord's Cricket Ground  
11                  11.72                  Lord's Cricket Ground  
12                  11.72                  Lord's Cricket Ground  
13                  11.71                  Lord's Cricket Ground  
14                  11.72                  Lord's Cricket Ground  
15                  11.72                  Lord's Cricket Ground  
16                  12.24          The County Ground, Eastbourne  
17                  12.18                  Forfar Cricket Ground  
18                  12.01         Grange Cricket Club, Edinburgh  
19                  12.01         Grange Cricket Club, Edinburgh  
20                  12.59          Malahide Cricket Club, Dublin

Final answer:                            Location  Latitude  Longitude  \
0                    Alnwick Castle   55.2466    -2.1790   
1                  Durham Cathedral   54.7737    -1.5727   
2              Gloucester Cathedral   51.8582    -2.2876   
3                      Lacock Abbey   51.2532    -1.6179   
4                     Christ Church   51.7567    -1.2580   
5                       New College   51.7544    -1.2489   
6               The Divinity School   51.7616    -1.2460   
7            Duke Humfrey's Library   51.7574    -1.2548   
8                        London Zoo   51.5373    -0.1419   
9                 Leadenhall Market   51.5121    -0.0945   
10                      Platform 9¾   51.5294    -0.1240   
11  Westminster Underground Station   51.5004    -0.1279   
12                        City Hall   51.5010    -0.1256   
13                Millennium Bridge   51.5038    -0.0988   
14                Piccadilly Circus   51.5108    -0.1403   
15                        Whitehall   51.5033    -0.1279   
16       Seven Sisters chalk cliffs   50.7917    -5.1965   
17                         Glen Coe   56.2514    -5.0513   
18              Greyfriars Kirkyard   55.9480    -3.1919   
19                        Edinburgh   55.9619    -3.1896   
20                  Cliffs of Moher   53.1496    -9.0605   

    Travel Time from Pune                 Nearby Cricket Stadium  
0                   11.92                  Bamburgh Cricket Club  
1                   11.86            Emirates Riverside (Durham)  
2                   11.93           St George's Park, Gloucester  
3                   11.87                 Salisbury Cricket Club  
4                   11.83  University Parks Cricket Club, Oxford  
5                   11.83  University Parks Cricket Club, Oxford  
6                   11.83  University Parks Cricket Club, Oxford  
7                   11.83  University Parks Cricket Club, Oxford  
8                   11.72                  Lord's Cricket Ground  
9                   11.71                  Lord's Cricket Ground  
10                  11.72                  Lord's Cricket Ground  
11                  11.72                  Lord's Cricket Ground  
12                  11.72                  Lord's Cricket Ground  
13                  11.71                  Lord's Cricket Ground  
14                  11.72                  Lord's Cricket Ground  
15                  11.72                  Lord's Cricket Ground  
16                  12.24          The County Ground, Eastbourne  
17                  12.18                  Forfar Cricket Ground  
18                  12.01         Grange Cricket Club, Edinburgh  
19                  12.01         Grange Cricket Club, Edinburgh  
20                  12.59          Malahide Cricket Club, Dublin  

[Step 7: Duration 17.99 seconds| Input tokens: 144,128 | Output tokens: 4,409]

In [14]:
result

,Location,Latitude,Longitude,Travel Time from Pune,Nearby Cricket Stadium
0,Alnwick Castle,55.2466,-2.1790,11.92,Bamburgh Cricket Club
1,Durham Cathedral,54.7737,-1.5727,11.86,Emirates Riverside (Durham)
2,Gloucester Cathedral,51.8582,-2.2876,11.93,"St George's Park, Gloucester"
3,Lacock Abbey,51.2532,-1.6179,11.87,Salisbury Cricket Club
4,Christ Church,51.7567,-1.2580,11.83,"University Parks Cricket Club, Oxford"
5,New College,51.7544,-1.2489,11.83,"University Parks Cricket Club, Oxford"
6,The Divinity School,51.7616,-1.2460,11.83,"University Parks Cricket Club, Oxford"
7,Duke Humfrey's Library,51.7574,-1.2548,11.83,"University Parks Cricket Club, Oxford"
8,London Zoo,51.5373,-0.1419,11.72,Lord's Cricket Ground
9,Leadenhall Market,51.5121,-0.0945,11.71,Lord's Cricket Ground


In [15]:
### Improving the results by further prompting.
agent.planning_interval = 4 ## after 4 steps agent will change its TAO with a new thought after 4 loops.

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.
{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey
### 1.1. Facts given in the task
- Origin coordinates (Pune): 18.5204° N, 73.8567° E

### 1.2. Facts to look up
- List of Harry Potter filming locations globally: We will need to perform a web search to gather these locations.
    - *Web search*: "Harry Potter filming locations"
- Coordinates of each Harry Potter filming location: We will need to find geographical coordinates of each location
found.
    - *Web sources*: Use the location names found from the previous search to get their exact coordinates from 
mapping websites such as Google Maps.
- Names of cricket stadiums within reasonable proximity to each Harry Potter filming location: We will need to 
identify cricket stadiums near each filming location using another set of web searches.
    - *Web search*: Query using the format "{location} cricket stadiums"

### 1.3. Facts to derive
- Travel times from each Harry Potter filming location to Pune via passenger plane: We will use the coordinates of 
both the filming locations and Pune to calculate these travel times.
    - *Calculation*: Perform computations using the `calculate_cargo_travel_time` tool to estimate the flight 
duration for a passenger plane assuming standard airspeeds.

---

## 2. Plan
1. Perform a web search for "Harry Potter filming locations".
2. Visit the source URLs from the web search results to confirm and compile the global list of Harry Potter filming
locations.
3. For each confirmed Harry Potter filming location, perform a web search to find its geographical coordinates.
4. Visit the source URLs confirming the geographical coordinates to ensure accuracy.
5. Perform a web search for cricket stadiums in proximity to each Harry Potter filming location to identify any 
nearby stadiums.
6. Visit the source URLs from the web search results for cricket stadiums to confirm the names and locations of 
stadiums nearby.
7. Use the `calculate_cargo_travel_time` tool to compute the travel times from each filming location to Pune via 
passenger plane.
8. Compile the data into a pandas dataframe including filming locations, their coordinates, nearby cricket 
stadiums, and the computed travel times.
9. Validate the data to ensure consistency and accuracy.
10. Provide the final answer using the `final_answer` tool, passing the compiled dataframe as input.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  hp_filming_locations_raw = web_search("Harry Potter filming locations")                                          
  print(hp_filming_locations_raw)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Harry Potter filming locations guide - VisitBritain 
Shop](https://www.visitbritainshop.com/us/en/harry-potter-filming-locations-guide)
Source: visitbritainshop.com

Where was Harry Potter filmed in England? · Durham Cathedral · Alnwick Castle · Malham Cove, Skipton · New College,
Oxford · Lacock Abbey · Visit Lacock Abbey ...See more

1. [Top Sights Where 'Harry Potter' Was 
Filmed](https://www.ricksteves.com/watch-read-listen/read/articles/harry-potter-sites)
Source: Rick Steves Europe

Many of the movies' exterior shots — especially scenes of the Hogwarts grounds — were filmed in the craggy, cloudy 
Highlands of Scotland (mostly in the Fort ...See more

2. [UK Harry Potter Filming Locations & 
Attractions](https://www.visitbritain.com/en/things-to-do/visit-harry-potter-filming-locations)
Source: Visit Britain

Harry Potter film locations around Britain · Freshwater West, Pembrokeshire, Wales · Lacock Abbey, Wiltshire, 
England · Alnwick Castle, Northumberland, England.See more

3. [31 Must Visit Harry Potter Filming Locations in the UK and 
...](https://flavourofthefilm.com/2024/11/01/31-must-visit-harry-potter-filming-locations-in-the-uk-and-ireland/)
Date published: Nov 1, 2024
Source: Flavour of the Film

Harry Potter Filming Locations: London · The Reptile House, London Zoo · Entrance to The Leaky Cauldron · Platform 
9 ¾, King's Cross Station.See more

4. [Places in Harry Potter](https://en.wikipedia.org/wiki/Places_in_Harry_Potter)
Source: Wikipedia

The Leaky Cauldron · Diagon Alley · Knockturn Alley · Platform Nine and Three-Quarters, King's Cross · Hogsmeade · 
La Place Cachée, Paris.See more

5. [The Complete Guide to Harry Potter Film Locations ...](https://londonmymind.com/harry-potter-film-locations/)
Date published: May 9, 2023
Source: London On My Mind

Harry Potter Film Locations in London · 1. Australia House · 2. Borough Market · 3. Claremont Square · 4. Great 
Scotland Place · 5. Lambeth ...See more

6. [Harry Potter at Alnwick 
Castle](https://www.alnwickcastle.com/about-alnwick-castle/alnwick-castle-on-screen/harry-potter-at-alnwick-castle)
Source: Alnwick Castle

For a day out at Hogwarts Castle, visit Alnwick Castle, an official filming location for Harry Potter & the 
Philosopher's Stone & Chamber of Secrets!

7. [Visit Harry Potter Filming Locations in the UK: A 
Guide](https://www.firebirdtours.com/blog/visit-harry-poter-filming-locations)
Date published: Feb 14, 2025
Source: Firebird Tours

Many Harry Potter scenes were filmed across the UK, with key locations including Alnwick Castle and Durham 
Cathedral for Hogwarts exteriors, ...See more

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Import necessary libraries                                                                                     
  import pandas as pd                                                                                              
  import math                                                                                                      
                                                                                                                   
  # Define origin coordinates for Pune as a tuple                                                                  
  origin_coords_pune = (18.5204, 73.8567)                                                                          
                                                                                                                   
  # Initialize lists to store data for DataFrame                                                                   
  locations = []                                                                                                   
  coords_lat = []                                                                                                  
  coords_lng = []                                                                                                  
  stadiums = []                                                                                                    
  travel_times = []                                                                                                
                                                                                                                   
  # Function to convert degrees to radians                                                                         
  def deg_to_rad(deg):                                                                                             
      return deg * (math.pi / 180)                                                                                 
                                                                                                                   
  # Calculate travel times from each filming location to Pune                                                      
  for location, coords in hp_filming_locations_coords.items():                                                     
      if coords is not None:                                                                                       
          dest_coords = (coords['lat'], coords['lng'])                                                             
          travel_time_minutes = calculate_cargo_travel_time(origin_coords=origin_coords_pune,                      
  destination_coords=dest_coords)                                                                                  
          travel_time_hours = round(travel_time_minutes / 60, 2)  # Convert to hours and round to 2 decimal        
  places                                                                                                           
          travel_times.append(travel_time_hours)                                                                   
      else:                                                                                                        
          travel_times.append(None)  # Assign None for locations with no coordinates                               
                                                                                                                   
      # Append data to respective lists                                                                            
      locations.append(location)                                                                                   
      coords_lat.append(coords['lat'] if coords else None)                                                         
      coords_lng.append(coords['lng'] if coords else None

Execution logs:
                                             Location   Latitude  Longitude  \
0              Entrance to The Leaky Cauldron, London  51.505440   0.091061   
1   Gringotts Wizarding Bank (filmed in Australia ...  51.512889  -0.125739   
2                            London City Hall, London  51.501364  -0.071351   
3      The Shire (filmed in Matlock Bath, Derbyshire)  53.158400  -1.509400   
4                           Millennium Bridge, London  51.510173   0.098438   
5                           Piccadilly Circus, London  51.510067   0.133869   
6       Godric's Hollow (filmed in Lavenham, Suffolk)  52.124200   1.283400   
7                           Leadenhall Market, London  51.512779   0.083662   
8                             Lacock Abbey, Wiltshire  51.414800  -1.740900   
9                            Claremont Square, London  51.530736   0.110208   
10                        Great Scotland Yard, London  51.506400   0.125600   
11                      The Reptile House, London Zoo  51.534806   0.154807   
12                    Durham Cathedral, County Durham  54.774720   1.576110   
13                           Hogwarts Castle Location  55.415558   1.705998   
14               St Pancras Renaissance Hotel, London  51.531000  -0.121000   
15              Freshwater West, Pembrokeshire, Wales  51.654987   5.058088   
16                     Alnwick Castle, Northumberland  55.415558   1.705998   
17  Hogwarts Express Route (filmed in Glenfinnan V...  56.876285   5.431914   
18              Gloucester Cathedral, Gloucestershire  51.867303   2.246668   
19  Hogsmeade Village (filmed in Hathersage, Derby...  53.256600  -1.454800   
20                       King's Cross Station, London  51.536100   0.125100   
21  Diagon Alley (external scenes, filmed in vario...  51.512779   0.083662   

                              Nearby Cricket Stadiums  \
0                                  Twickenham Stadium   
1                     No nearby cricket stadium found   
2                                  Twickenham Stadium   
3                                County Ground, Derby   
4                                  Twickenham Stadium   
5                                  Twickenham Stadium   
6                     No nearby cricket stadium found   
7   The Den Stadium, The Emirates Stadium, Olympic...   
8                              County Ground, Bristol   
9                     No nearby cricket stadium found   
10                              Lord's Cricket Ground   
11                              Lord's Cricket Ground   
12                      Banks Homes Riverside Stadium   
13                           Newcastle United Stadium   
14                                   Emirates Stadium   
15                              Bridge Meadow Stadium   
16                    No nearby cricket stadium found   
17                    No nearby cricket stadium found   
18  The Gloucestershire County Cricket Club is a p...   
19                    No nearby cricket stadium found   
20                    No nearby cricket stadium found   
21                    No nearby cricket stadium found   

    Travel Time to Pune (hours)  
0                          0.19  
1                          0.20  
2                          0.20  
3                          0.20  
4                          0.19  
5                          0.19  
6                          0.19  
7                          0.19  
8                          0.20  
9                          0.19  
10                         0.19  
11                         0.19  
12                         0.19  
13                         0.19  
14                         0.20  
15                         0.19  
16                         0.19  
17                         0.19  
18                         0.19  
19                         0.20  
20                         0.19  
21                         0.19

Final answer:                                              Location   Latitude  Longitude  \
0              Entrance to The Leaky Cauldron, London  51.505440   0.091061   
1   Gringotts Wizarding Bank (filmed in Australia ...  51.512889  -0.125739   
2                            London City Hall, London  51.501364  -0.071351   
3      The Shire (filmed in Matlock Bath, Derbyshire)  53.158400  -1.509400   
4                           Millennium Bridge, London  51.510173   0.098438   
5                           Piccadilly Circus, London  51.510067   0.133869   
6       Godric's Hollow (filmed in Lavenham, Suffolk)  52.124200   1.283400   
7                           Leadenhall Market, London  51.512779   0.083662   
8                             Lacock Abbey, Wiltshire  51.414800  -1.740900   
9                            Claremont Square, London  51.530736   0.110208   
10                        Great Scotland Yard, London  51.506400   0.125600   
11                      The Reptile House, London Zoo  51.534806   0.154807   
12                    Durham Cathedral, County Durham  54.774720   1.576110   
13                           Hogwarts Castle Location  55.415558   1.705998   
14               St Pancras Renaissance Hotel, London  51.531000  -0.121000   
15              Freshwater West, Pembrokeshire, Wales  51.654987   5.058088   
16                     Alnwick Castle, Northumberland  55.415558   1.705998   
17  Hogwarts Express Route (filmed in Glenfinnan V...  56.876285   5.431914   
18              Gloucester Cathedral, Gloucestershire  51.867303   2.246668   
19  Hogsmeade Village (filmed in Hathersage, Derby...  53.256600  -1.454800   
20                       King's Cross Station, London  51.536100   0.125100   
21  Diagon Alley (external scenes, filmed in vario...  51.512779   0.083662   

                              Nearby Cricket Stadiums  \
0                                  Twickenham Stadium   
1                     No nearby cricket stadium found   
2                                  Twickenham Stadium   
3                                County Ground, Derby   
4                                  Twickenham Stadium   
5                                  Twickenham Stadium   
6                     No nearby cricket stadium found   
7   The Den Stadium, The Emirates Stadium, Olympic...   
8                              County Ground, Bristol   
9                     No nearby cricket stadium found   
10                              Lord's Cricket Ground   
11                              Lord's Cricket Ground   
12                      Banks Homes Riverside Stadium   
13                           Newcastle United Stadium   
14                                   Emirates Stadium   
15                              Bridge Meadow Stadium   
16                    No nearby cricket stadium found   
17                    No nearby cricket stadium found   
18  The Gloucestershire County Cricket Club is a p...   
19                    No nearby cricket stadium found   
20                    No nearby cricket stadium found   
21                    No nearby cricket stadium found   

    Travel Time to Pune (hours)  
0                          0.19  
1                          0.20  
2                          0.20  
3                          0.20  
4                          0.19  
5                          0.19  
6                          0.19  
7                          0.19  
8                          0.20  
9                          0.19  
10                         0.19  
11                         0.19  
12                         0.19  
13                         0.19  
14                         0.20  
15                         0.19  
16                         0.19  
17                         0.19  
18                         0.19  
19                         0.20  
20                         0.19  
21                         0.19  

[Step 10: Duration 10.26 seconds| Input tokens: 227,584 | Output tokens: 3,942]

                                             Location   Latitude  Longitude  \
0              Entrance to The Leaky Cauldron, London  51.505440   0.091061   
1   Gringotts Wizarding Bank (filmed in Australia ...  51.512889  -0.125739   
2                            London City Hall, London  51.501364  -0.071351   
3      The Shire (filmed in Matlock Bath, Derbyshire)  53.158400  -1.509400   
4                           Millennium Bridge, London  51.510173   0.098438   
5                           Piccadilly Circus, London  51.510067   0.133869   
6       Godric's Hollow (filmed in Lavenham, Suffolk)  52.124200   1.283400   
7                           Leadenhall Market, London  51.512779   0.083662   
8                             Lacock Abbey, Wiltshire  51.414800  -1.740900   
9                            Claremont Square, London  51.530736   0.110208   
10                        Great Scotland Yard, London  51.506400   0.125600   
11                      The Reptile House, London Zo

In [16]:
detailed_report

,Location,Latitude,Longitude,Nearby Cricket Stadiums,Travel Time to Pune (hours)
0,"Entrance to The Leaky Cauldron, London",51.505440,0.091061,Twickenham Stadium,0.19
1,Gringotts Wizarding Bank (filmed in Australia ...,51.512889,-0.125739,No nearby cricket stadium found,0.20
2,"London City Hall, London",51.501364,-0.071351,Twickenham Stadium,0.20
3,"The Shire (filmed in Matlock Bath, Derbyshire)",53.158400,-1.509400,"County Ground, Derby",0.20
4,"Millennium Bridge, London",51.510173,0.098438,Twickenham Stadium,0.19
5,"Piccadilly Circus, London",51.510067,0.133869,Twickenham Stadium,0.19
6,"Godric's Hollow (filmed in Lavenham, Suffolk)",52.124200,1.283400,No nearby cricket stadium found,0.19
7,"Leadenhall Market, London",51.512779,0.083662,"The Den Stadium, The Emirates Stadium, Olympic...",0.19
8,"Lacock Abbey, Wiltshire",51.414800,-1.740900,"County Ground, Bristol",0.20
9,"Claremont Square, London",51.530736,0.110208,No nearby cricket stadium found,0.19


### Splitting one task into multiple agents so that one agent is focused on one task for better accuracy.

In [17]:
os.environ["SERPER_API_KEY"] = ""

model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [18]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = ""

In [19]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(

model=OpenAIServerModel("gpt-4o", max_tokens=8096),
   tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
    "geopandas",
    "plotly",
    "plotly.io",  # explicitly add this
    "plotly.express",  # explicitly add this too
    "shapely",
    "json",
    "pandas",
    "numpy",
    "kaleido"
],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [20]:
manager_agent.visualize()

CodeAgent | gpt-4o
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.io', 'plotly.express', 'shapely', 'json', 'pandas', 
│   'numpy', 'kaleido']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │               

In [22]:
!pip install -U kaleido

In [23]:
manager_agent.run("""
ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here │
│ (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                           │
│ Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points │
│ in total.                                                                                                       │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Current location is Pune with coordinates (18.5204° N, 73.8567° E).
- There is a need to find Harry Potter filming locations.
- The travel time is calculated using a passenger plane.
- There is a need to find nearby cricket stadiums to the filming locations.
- Locations need to be placed on a spatial map where points are colored based on travel time.

### 1.2. Facts to look up
- The list of all Harry Potter filming locations worldwide, along with their coordinates.
- Information on nearby cricket stadiums to each Harry Potter filming location and their coordinates.
- The typical cruising speed of passenger airplanes for calculating travel times.
- Sources for filming locations could include websites containing film location information, such as fan sites or 
movie databases.
- Sources for cricket stadiums can be sports websites or databases that list sports venues by location.

### 1.3. Facts to derive
- The travel time from Pune to each Harry Potter filming location using the coordinates and the typical passenger 
plane cruising speed.
- Identification of a minimum of six spatial points (combined Harry Potter filming locations and cricket stadiums).
- The map’s color scale corresponding to the calculated travel times.

## 2. Plan

1. **Look up Harry Potter filming locations:**
   - Utilize the web agent to compile a comprehensive list of Harry Potter filming locations around the world, 
including their geographic coordinates.

2. **Find Nearby Cricket Stadiums:**
   - For each Harry Potter filming location, use the web agent to identify nearby cricket stadiums, preferably 
within a reasonable distance, and gather their coordinates.

3. **Determine Travel Times:**
   - Use the `calculate_cargo_travel_time` function to calculate the travel time from Pune to each filming location
and cricket stadium using the cruising speed of a typical passenger airplane.

4. **Verify Point Count:**
   - Ensure that there are at least six spatial points when combining Harry Potter filming locations and cricket 
stadiums.

5. **Build Pandas DataFrame:**
   - Construct a pandas DataFrame containing the names, coordinates (latitude and longitude), and calculated travel
times for all valid points.

6. **Plot Data on Global Map:**
   - Use plotly to create a scatter plot on a world map with each point color-coded according to the time it takes 
to travel from Pune. Each point should be labeled clearly.

7. **Save and Display the Map:**
   - Save the map as 'saved_map.png' and present it.

8. **Return the final outcome:**
    - Use the final_answer function to provide the visualized map along with any required data back to the 
requester.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
To complete the plan, I'll go sequentially through the steps outlined before. Let's start with looking up the Harry
Potter filming locations.                                                                                          
                                                                                                                   
### Step 1: Look up Harry Potter filming locations                                                                 
                                                                                                                   
I will use the web agent to find a comprehensive list of Harry Potter filming locations around the world, including
their geographic coordinates.                                                                                      
                                                                                                                   
<code>                                                                                                             
harry_potter_locations_task = """                                                                                  
Find a comprehensive list of all Harry Potter filming locations worldwide. For each location, retrieve its name,   
corresponding geographical coordinates (latitude and longitude), and any significant details, if available. Ensure 
the information is extracted accurately from reliable sources.                                                     
"""                                                                                                                
                                                                                                                   
harry_potter_locations_info = web_agent(task=harry_potter_locations_task, additional_args={})                      
print(harry_potter_locations_info)                                                                                 
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  harry_potter_locations_task = """                                                                                
  Find a comprehensive list of all Harry Potter filming locations worldwide. For each location, retrieve its       
  name, corresponding geographical coordinates (latitude and longitude), and any significant details, if           
  available. Ensure the information is extracted accurately from reliable sources.                                 
  """                                                                                                              
                                                                                                                   
  harry_potter_locations_info = web_agent(task=harry_potter_locations_task, additional_args={})                    
  print(harry_potter_locations_info)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Create a scatter plot map using Plotly Express without using restricted color scales                           
  fig = px.scatter_geo(df_locations,                                                                               
                       lat='latitude',                                                                             
                       lon='longitude',                                                                            
                       text='name',                                                                                
                       color='travel_time',                                                                        
                       # Use named colors or basic color scale like 'Blues'                                        
                       color_continuous_scale='Blues',                                                             
                       size_max=15,                                                                                
                       title='Harry Potter Filming Locations and Cricket Stadiums Travel Time from Pune',          
                       projection='natural earth')                                                                 
                                                                                                                   
  # Show the figure                                                                                                
  fig.show()                                                                                                       
                                                                                                                   
  # Since saving as an image is restricted, output the figure directly                                             
  final_answer(fig)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Figure({
    'data': [{'geo': 'geo',
              'hovertemplate': ('name=%{text}<br>latitude=%{lat' ... '%{marker.color}<extra></extra>'),
              'lat': array([51.752 , 56.8214, 56.8714, 55.4151, 51.5283, 52.4556]),
              'legendgroup': '',
              'lon': array([-1.2577, -5.4385, -5.4344, -1.7055, -0.1721, -1.9036]),
              'marker': {'color': array([8.20446022, 8.49553484, 8.49508409, 8.23953753, 8.12120233, 8.25281438]),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers+text',
              'name': '',
              'showlegend': False,
              'text': array(['Oxford', 'Loch Shiel', 'Glenfinnan Viaduct', 'Alnwick Castle',
                             "Lord's Cricket Ground", 'Edgbaston Cricket Ground'], dtype=object),
              'type': 'scattergeo'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'travel_time'}},
                             'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                            'rgb(222,235,247)'], [0.25,
                                            'rgb(198,219,239)'], [0.375,
                                            'rgb(158,202,225)'], [0.5,
                                            'rgb(107,174,214)'], [0.625,
                                            'rgb(66,146,198)'], [0.75,
                                            'rgb(33,113,181)'], [0.875,
                                            'rgb(8,81,156)'], [1.0,
                                            'rgb(8,48,107)']]},
               'geo': {'center': {},
                       'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                       'projection': {'type': 'natural earth'}},
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Harry Potter Filming Locations and Cricket Stadiums Travel Time from Pune'}}
})

Check check_reasoning_and_plot failed with error: Make sure to save the plot under saved_map.png!

[Step 10: Duration 4.48 seconds| Input tokens: 62,432 | Output tokens: 2,450]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-rLs8S5duoHbsBhVgzAUAivdU on tokens per min (TPM): Limit 30000, Used 25932, Requested 5154. Please try again in 2.172s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [24]:
manager_agent.python_executor.state["fig"]